# Scooter sharing in Austin, Texas

In many cities around the world, micromobility in the form of
electric scooters has recently become very popular.  The city of
Austin, Texas has made available trip-level data from all scooter
companies operating within the city limits.

This is the first notebook that we see in Stats 206.  We are mainly
focused here on introducing the software tools and doing some very
basic summarization and description of the data.  There are a lot of
new programming concepts below, and some of them are deliberately
only explained at a high level.  We will revisit many of these
concepts later in the course and discuss them in greater detail at
that time.

# Importing modules

Python is a general-purpose programming language.  On its own, it
can be used for some basic tasks.  Most of the time, when working
with Python, you will be importing _modules_ containing code that is
not part of the core Python language, that will help you perform
more specialized tasks.  For scientific computing and data science,
we almost always import the numpy and pandas modules.

In [ ]:
import numpy as np
import pandas as pd

In addition, we will often need the "os" (operating system) module:

In [ ]:
import os

# Loading data from a file

The scooter data are in a format called "csv", which stands for
"comma separated values".  This is the most common portable
text-based format for rectangular data (data arranged in a table
with rows and columns).  The "pandas" package, imported above,
provides data management and basic data analysis capabilities in
Python.  The central element of Pandas is a "dataframe", which is a
rectangular structure for holding data.

Our first step here is to read the data from a csv format file
containing the scooter trips, and represent it in the computer's
memory as a dataframe.

First, modify this string according to your section number (001 or
002):

In [ ]:
f = "stats206s002f21"

Now Pandas can read the data using the line of code below:

In [ ]:
base = "/scratch/%s_class_root/%s_class/shared_data/datasets" % (f, f)
df = pd.read_csv(os.path.join(base, "scooters_short.csv.gz"))

The above commands create a dataframe holding the scooter trip data,
and creates a variable named `df` that is bound to this value.

In the above commands, the values enclosed in quotation marks are
"strings".  A string is a value that contains a sequence of text or
characters.  In this case, the string bound to the "base" variable
is the "file path" pointing to the CSV file on greatlakes that we
wish to load.

Note also that the filename above ends with ".gz".  This indicates
that the data are compressed so that the file takes up less space on
the disk.  Pandas will automatically decompress the file when
reading it.

The dataframe that we just loaded contains all trips for a random
selection of 20,000 scooters drawn from the complete Austin mobility
dataset.

# Getting some basic information about the data

Now that we have the dataset loaded, we can do some basic things
with it.  First, we will get the shape, or dimensions of the
dataframe:

In [ ]:
df.shape

The shape attribute contains the number of rows and the number of
columns in the dataframe.  This dataset is a subset containing about
one quarter of the complete Austin e-mobility dataset.

Like most rectangular datasets, the rows of this dataset represent
cases and the columns represent variables.  A case here is a single
trip on a rented scooter.  The columns (variables) provide
information about the trip.  Next we will see what the names of the
variables are.

In [ ]:
df.columns

Some of these variable names are self-explanatory, for others, we
will explain their meaning below as needed.  The next line of code
displays the first few rows of the dataframe.  Depending on the
width of your screen, it is possible that only a subset of the
columns will be shown, in which case the omitted columns are
displayed as "...".

In [ ]:
df.head()

Note that in the preceeding two code cells, "head" is followed by
parentheses and "columns" is not. This is because "head" is a method
while "columns" is an attribute.  Methods are essentially functions,
and functions take arguments inside a pair of parentheses.
Attributes are not functions, and therefore do not have parentheses.
It is an error to use parentheses on an attribute, or to omit the
parentheses on a method.  To avoid such errors, you will need to
learn when you have an attribute and when you have a method.

The `head` method takes an optional argument, which is the number of
rows to display.  The default value of this argument is 5, so if we
call the method without any arguments, like above, we see five rows
of data.  If we want to see a different number of rows, we can pass
an argument like this:

In [ ]:
df.head(3)

The "device ID" is a unique identifier for each scooter. The number
of times that a device ID appears in the dataframe is the number of
times that the scooter was rented during the period of time covered
by the dataset.  Most of the device IDs appear multiple times in the
data set, since most of the scooters were rented multiple times.

To learn more about the device IDs, we can select the column of the
dataframe containing these values using the syntax `df["Device
ID"]`.  This creates a "series" since it contains data for only one
variable. The `unique` method then constructs another series
containing only the unique device IDs, and the `size` attribute of
this series tells us how many elements it contains.  Thus, the
following line of code tells us how many unique scooters are
represented in this dataset.

In [ ]:
df["Device ID"].unique().size

The above code uses "chaining" to do three things in one line of
code.  It may be instructive to see how this can be done in three
lines, without chaining.

In [ ]:
x = df["Device ID"]
y = x.unique()
y.size

We can now look at some summary statistics of how many times each
scooter was rented.  The `value_counts` method determines all of the
distinct values in a series and counts how many times each occurs.
The `describe` method then produces a set of summary statistics for
these counts.

In [ ]:
df["Device ID"].value_counts().describe()

When Pandas reads a CSV file, the date and time variables are
usually not automatically converted to proper Pandas format.  The
`dtypes` attribute tells us the storage format of each variable in
the dataframe.  We can see below that the "Start Time" and "End
Time" variables have `object` storage format. To work with these
variables as time values we will want to convert them to "datetime64"
values.

In [ ]:
df.dtypes

The following lines of code perform this conversion for the "Start
Time" variable.  Don't worry about the details of this conversion at
the moment.

In [ ]:
f = "%m/%d/%Y %I:%M:%S %p"
df["Start Time"] = pd.to_datetime(df["Start Time"], format=f)

Now we check the types again and see that `Start Time` has type
"datetime64".

In [ ]:
df.dtypes

Now that we have this variable in a proper format, we can determine
the period of time covered by this dataset.

In [ ]:
print(df["Start Time"].min())
print(df["Start Time"].max())

Regarding the use of `print` above -- the `print` function is used
to print a value to the notebook output cell.  By default, the value
of the last expression in any code cell is always printed.  If you
want to print more than one value from within a single code cell,
you should use the `print` function.

## Scooter lifespan

One question that is often asked about scooter rentals is how long
the scooters stay in the fleet.  In some cities, scooters have
generally not lasted very long before breaking or being stolen or
lost.  To address this question, we can look at the earliest and
last rental of each scooter in the dataset.  We do this by creating
a "grouped dataframe" with the `groupby` method.  Grouped dataframes
are very powerful and we will use them frequently in this course.
We will explain how grouped dataframes work in more detail later.
This example is an introduction to show how grouped dataframes can
be used to achieve our current goals.

In [ ]:
dx = df.groupby("Device ID")["Start Time"].agg([np.min, np.max])
print(dx.columns)
dx["duration"] = dx["amax"] - dx["amin"]
print(df.shape, dx.shape)
dx["duration"].describe()

`groupby` carries out a stratification of the data.  In this case,
we are stratifying the data by the `Device ID` variable.  Every
distinct value of `Device ID` forms a group.  A group contains all
trips recorded for one scooter.  We then select only the `Start
Time` variable.  Finally, the `agg` method is used to carry out an
aggregation or summarization of the data for each group.

The functions passed as arguments to `agg()` are the functions used
to do the aggregation.  Here we are aggregating using the `min` and
`max` functions. These will give us the minimum (earliest) and
maximum (latest) trip for each scooter.  The `np.` is placed in
front of the `min` and `max` functions because they are part of the
numpy package, which we imported above.

Sometimes it can be hard to know what names the variables created by
the aggregation will have.  Therefore, we print the names and see
here that they are "amax" and "amin".

We also printed the shapes of the two dataframes `df` and `dx`.
Here, `df` is the original dataframe that we loaded from a csv file,
and `dx` is the summarized dataframe containing one row for each
distinct device ID.  Since there are 20,000 scooters in the dataset,
this summarized dataframe contains exactly 20,000 rows.

Once we have the minimum and maximum rental date per scooter, we can
subtract them to get the duration of time between the first and last
rental.  Then, we use the `describe` method to get some basic
summary information about these durations.

We see that the median scooter was used for a period of 82 days.
Note that this is a biased estimate of the actual median lifespan
due to "truncation" and "censoring".  We won't define these terms
precisely here, but they refer to the fact that some scooters were
in use before and/or after the period of time covered by this
dataset, so their actual lifespan is longer than the value
calculated here.  There are more advanced statistical methods that
can be used to correct for this bias, but we will not pursue that
here.

## Scooter usage over time

The usage of scooters varies throughout the year, and over the span
of several years.  Scooter usage also varies by time of day and day
of week, but this is a separate topic that we will revisit later.
To focus on the longer term variation in scooter usage, we create a
variable below that counts the total number of trips taken during
each week within the dataset.  We have around two year's of data, so
there are slightly more than 100 weeks.

To accomplish this goal, we will add two new columns to the
dataframe, one containing the year in which the trip occured, and
one containing the week within the year that the trip occured (which
is an integer between 1 and 52).  Since we already have the start
time of each trip, we only need to exract the year and week from the
start time, as shown below.  The `dt` symbol is a special attribute
of datetime variables that allows us to do things that only make
sense with date and/or time values.

In [ ]:
df["year"] = df["Start Time"].dt.year
df["weekofyear"] = df["Start Time"].dt.isocalendar().week

Now that we have a variables indicating the year and week of each
trip, we can use `groupby` to count how many trips occur for each
combination of a year and a week.  Note that here, unlike above, we
are grouping by two variables, which must be placed in a list using
square brackets (`[]`).

In [ ]:
dx = df.groupby(["year", "weekofyear"]).size()

Here we are taking advantage of the built in `size` aggregation
method.  This gives us the number of rows per group, which is
equivalent to the number of trips per week.  The same result can be
obtained using

In [ ]:
dxx = df.groupby(["year", "weekofyear"])["Device ID"].agg(np.size)

To confirm that these two series are equal, we can use the
following:

In [ ]:
assert(all(dx == dxx))

It would make most sense to graph these counts, but we are not
introducing graphing yet in this notebook, so we will simply display
the counts in a table.  Since our grouped dataframe here groups by
two variables, we have what is called a "hierarchical index".  We
could view the result (`dx`) directly, but to make it easier to read
the table, we can "unstack" it to produce a table in which the rows
are years and the columns are weeks.  This will be a 4 x 52 table
since we have at least some data in each of four years.  For viewing
on the screen, it is better to have a table that is longer rather
than wider, so we use the transpose (`T`) attribute to put the weeks
in the rows and the years in the columns.

In [ ]:
dx.unstack().T

In the result of the above cell, a `NaN` ("not a number") will
appear in any cell that has no data.  This includes some cells that
fall in the future relative to when the data were obtained, and some
earlier cells for which no data are available.

The results show that scooter usage increased rapidly in late summer
of 2018 and remained very popular until late fall of 2019.  Note the
spike in week 11 of 2019, corresponding to the "South by Southwest"
event.  By late fall of 2019 and into the early winter of 2020,
scooter usage had dropped by around half relative to the same period
in the previous year.  Then around week 12 of 2020, the COVID
pandemic reached the US, and scooter usage plumetted (although not
to zero).  By the end of the data series in the summer of 2020,
usage had recovered slightly, but only to around 15% of usage in the
previous year.  In 2021, scooter usage remained somewhat lower than
pre-pandemic levels.

## Trip duration

Another characteristic of interest is the length of time of each
scooter trip. The `Trip Duration` variable contains this information
in units of seconds.  Suppose we wish to calculate some basic 
summary statistics for the trip durations for trips taken each 
month.  The `describe` method calculates many common summary
statistics.

In [ ]:
df.groupby("Month")["Trip Duration"].describe()

If we want to calculate a particular summary statistic, for example
several quantiles, we can proceed as below.  Note that by dividing
the results by 60, the units become minutes rather than seconds.

In [ ]:
df.groupby("Month")["Trip Duration"].quantile([0.5, 0.9]).unstack() / 60